In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import torch
import open_clip
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm 
from timm import create_model
from torchvision import transforms
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [3]:
file_path = "/users/PAS2136/rayees/CV4A - Efficient Probing/ProbingSubsets/probing_5800_ratio.csv"
df = pd.read_csv(file_path)
cols = ['ImageFilePath', 'ScientificName']
df = df[cols]
df = df.dropna()
df.head(2)

,ImageFilePath,ScientificName
0,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Britis...,Platyderus depressus
1,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Britis...,Nebria brevicollis


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = create_model("convnext_base", pretrained=True).to(device)


preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [5]:
def extract_features(image_path) : 
    
    image = Image.open(image_path).convert("RGB")
    
    image_tensor = preprocess(image).unsqueeze(0).to(device)

    with torch.no_grad():
        features = model.forward_features(image_tensor).mean(dim=[2, 3])

    return features.cpu().numpy()
    

In [6]:
X = np.vstack([extract_features(img) for img in tqdm(df["ImageFilePath"], desc="Processing Images ...") ])

seed = 99

le = LabelEncoder()

y = le.fit_transform(df["ScientificName"])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(
    X, y, df_indices, 
    test_size=0.2, 
    random_state=seed,
    stratify=y
)


Processing Images ...:   0%|          | 0/5800 [00:00<?, ?it/s]

In [7]:

test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


In [8]:

models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=11),  
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=seed)
}

predictions = {}

metrics = {}


for name, model in tqdm(models.items(), desc="Training Models", leave=True):
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds

    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)

    metrics[name] = {"Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}

    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


Training Models:   0%|          | 0/4 [00:00<?, ?it/s]

NaiveBayes                | Acc: 47.33% | Prec: 47.25% | Rec: 47.33% | F1: 45.28% | Bal Acc: 39.32% | MCC: 0.4706
LogisticRegression        | Acc: 70.00% | Prec: 69.68% | Rec: 70.00% | F1: 68.09% | Bal Acc: 62.65% | MCC: 0.6985
NearestNeighbor           | Acc: 44.91% | Prec: 45.46% | Rec: 44.91% | F1: 42.13% | Bal Acc: 35.92% | MCC: 0.4463
MLP_Baseline              | Acc: 63.71% | Prec: 63.35% | Rec: 63.71% | F1: 61.74% | Bal Acc: 56.26% | MCC: 0.6353


In [9]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,ScientificName,Pred_NaiveBayes,Pred_LogisticRegression,Pred_NearestNeighbor,Pred_MLP_Baseline
3127,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Britis...,Pterostichus diligens,Pterostichus minor,Pterostichus minor,Pterostichus vernalis,Pterostichus minor
3164,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Britis...,Amara consularis,Amara apricaria,Amara plebeja,Amara apricaria,Amara plebeja


In [10]:

basedir = '/users/PAS2136/rayees/CV4A - Efficient Probing/BritishCarabids_5800_ratio/'


In [11]:

test_df.to_csv(f"{basedir}/4.ConvNeXt-species.csv", index=False)

metrics_df.to_csv(f"{basedir}/4.ConvNeXt-species-metrics.csv", index=False)

